In [7]:
# Import all the necessary libraries
import pandas as pd
import numpy as np
from pyomo.environ import *
from pyomo.core.expr.numeric_expr import sqrt

In [8]:
# Purpose of the code is to minimize the exposure to healthy tissue while ensuring that each tumor tissue cell receives at least 3 units of exposure. 
# We want to search for the optimal seed positions will ensure this. 

# Define the model
model = ConcreteModel()

# Here we define the set of healthy tissue. The coordiantes represent the center of each tissue cell. 
model.Healthy_Centers = Set(initialize=[(1,4), (1,3), (1,1), (2,4), (2,1), (3,4), (3,2), (3,1), (4,4), (4,3), (4,2)])
# Here we define the set of tumor tissue. The coordiantes represent the center of each tissue cell.
model.Tumor_Centers = Set(initialize=[(1,2), (2,2), (2,3), (3,3), (4,1)])
# Here we define the set of seeds. We can only have 3 seeds total. This essentially acts as a constraint on the number of seeds we can use.
model.K = RangeSet(3)

# Here we define the continuous variables for the seed positions. We set the bounds to be between 0 and 4 based on the grid size.
model.SeedsX = Var(model.K, within=NonNegativeReals, bounds=(0, 4))
model.SeedsY = Var(model.K, within=NonNegativeReals, bounds=(0, 4))

# Here we define implicit variables that represent the Euclidean distance between each healthy tissue cell and each seed. 
# Here we create a variable for each combination of healthy tissue center and seed.
model.EuclideanDistanceHealthy = Var(model.Healthy_Centers, model.K, within=NonNegativeReals)
# Here we define implicit variables that represent the Euclidean distance between each tumor tissue cell and each seed.
model.EuclideanDistanceTumor = Var(model.Tumor_Centers, model.K, within=NonNegativeReals)

# These are functions that define how we can calculate the distance between each tissue cell and each seed.
def distance_healthy_rule(model, i, j, k):
    """
    This function defines the Euclidean distance between each healthy tissue cell and each seed.
    Model - The model object that contains all model information such as sets, variables, and constraints.
    i - The x-coordinate of the healthy tissue cell.
    j - The y-coordinate of the healthy tissue cell.
    k - The seed number.
    Returns the Euclidean distance between the healthy tissue cell and the seed.
    """
    return model.EuclideanDistanceHealthy[(i, j), k] == sqrt((i - model.SeedsX[k])**2 + (j - model.SeedsY[k])**2)
def distance_tumor_rule(model, i, j, k):
    """
    This function defines the Euclidean distance between each tumor tissue cell and each seed.
    Model - The model object that contains all model information such as sets, variables, and constraints.
    i - The x-coordinate of the tumor tissue cell.
    j - The y-coordinate of the tumor tissue cell.
    k - The seed number.
    Returns the Euclidean distance between the tumor tissue cell and the seed.
    """
    return model.EuclideanDistanceTumor[(i, j), k] == sqrt((i - model.SeedsX[k])**2 + (j - model.SeedsY[k])**2)


# Here we apply the functions for the combination of healthy tissue cells and seeds, and tumor tissue cells and seeds.
# Here we multiply the number of healthy tissue cells by the number of seeds so that we can create a list containing the coordinates of each healthy tissue cell and each seed.
# Effectively, we are creating the cartesian product of the healthy tissue cells and the seeds so that we can apply the distance_healthy_rule function to each combination.
model.DistanceHealthyConstraint = Constraint(model.Healthy_Centers * model.K, rule=distance_healthy_rule)
model.DistanceTumorConstraint = Constraint(model.Tumor_Centers * model.K, rule=distance_tumor_rule)

# The objective function's primary purpose is to minimize the exposure to healthy tissue
def objective_minimize(model):
    # Here we calculate the exposure to healthy tissue by summing the inverse of the Euclidean distance + 0.01 between each healthy tissue cell and each seed.
    return sum(1 / (model.EuclideanDistanceHealthy[h, k] + 0.01) for h in model.Healthy_Centers for k in model.K)
model.objective = Objective(rule=objective_minimize, sense=minimize)

# Here our objective is to ensure that each tumor cell receives at least 3 units of exposure.
def tumor_constraint_rule(model, i, j):
    # Here we calculate the exposure to tumor tissue by summing the inverse of the Euclidean distance + 0.01 between each tumor tissue cell and each seed.
    return sum(1 / (model.EuclideanDistanceTumor[(i, j), k] + 0.01) for k in model.K) >= 3
model.Tumor_Constraints = Constraint(model.Tumor_Centers, rule=tumor_constraint_rule)

# Setup the solver
solver = SolverFactory('ipopt')

# Get the answer! 
Optimal_Seeds = solver.solve(model, tee=True)

Ipopt 3.14.11: 

******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit https://github.com/coin-or/Ipopt
******************************************************************************

This is Ipopt version 3.14.11, running with linear solver MUMPS 5.2.1.

Number of nonzeros in equality constraint Jacobian...:      144
Number of nonzeros in inequality constraint Jacobian.:       15
Number of nonzeros in Lagrangian Hessian.............:       57

Total number of variables............................:       54
                     variables with only lower bounds:       48
                variables with lower and upper bounds:        6
                     variables with only upper bounds:        0
Total number of equality constraints.................:       48
Total number

In [9]:
# Here we print the optimal seed coordinates
for seed in model.K:
    print('The optimal seed coordinates are:')
    print(f"Seed {seed} X-coordinate: {model.SeedsX[seed].value}")
    print(f"Seed {seed} Y-coordinate: {model.SeedsY[seed].value}")

The optimal seed coordinates are:
Seed 1 X-coordinate: 3.999999856088785
Seed 1 Y-coordinate: 0.5615297428060256
The optimal seed coordinates are:
Seed 2 X-coordinate: 2.5382686746816936
Seed 2 Y-coordinate: 2.9155865885944814
The optimal seed coordinates are:
Seed 3 X-coordinate: 1.410220244131618
Seed 3 Y-coordinate: 2.042847847111256
